In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re

In [ ]:

stop_word_dic={}
with open('sample_data/extra_stop.txt','r') as file:
   
    # reading each line    
    for line in file:
   
        # reading each word        
        for word in line.split():
   
            # displaying the words           
            stop_word_dic[word]=word
with open('sample_data/stop_words_english.txt','r') as file:
   
    # reading each line    
    for line in file:
   
        # reading each word        
        for word in line.split():
   
            # displaying the words           
            stop_word_dic[word]=word



closed_class_stop_words = ['a', 'the', 'an', 'and', 'or', 'but', 'about', 'above', 'after', 'along', 'amid', 'among', \
                           'as', 'at', 'by', 'for', 'from', 'in', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', \
                           'onto', 'out', 'over', 'past', 'per', 'plus', 'since', 'till', 'to', 'under', 'until', 'up', \
                           'via', 'vs', 'with', 'that', 'can', 'cannot', 'could', 'may', 'might', 'must', \
                           'need', 'ought', 'shall', 'should', 'will', 'would', 'have', 'had', 'has', 'having', 'be', \
                           'is', 'am', 'are', 'was', 'were', 'being', 'been', 'get', 'gets', 'got', 'gotten', \
                           'getting', 'seem', 'seeming', 'seems', 'seemed', \
                           'enough', 'both', 'all', 'your' 'those', 'this', 'these', \
                           'their', 'the', 'that', 'some', 'our', 'no', 'neither', 'my', \
                           'its', 'his' 'her', 'every', 'either', 'each', 'any', 'another', \
                           'an', 'a', 'just', 'mere', 'such', 'merely' 'right', 'no', 'not', \
                           'only', 'sheer', 'even', 'especially', 'namely', 'as', 'more', \
                           'most', 'less' 'least', 'so', 'enough', 'too', 'pretty', 'quite', \
                           'rather', 'somewhat', 'sufficiently' 'same', 'different', 'such', \
                           'when', 'why', 'where', 'how', 'what', 'who', 'whom', 'which', \
                           'whether', 'why', 'whose', 'if', 'anybody', 'anyone', 'anyplace', \
                           'anything', 'anytime' 'anywhere', 'everybody', 'everyday', \
                           'everyone', 'everyplace', 'everything' 'everywhere', 'whatever', \
                           'whenever', 'whereever', 'whichever', 'whoever', 'whomever' 'he', \
                           'him', 'his', 'her', 'she', 'it', 'they', 'them', 'its', 'their', 'theirs', \
                           'you', 'your', 'yours', 'me', 'my', 'mine', 'I', 'we', 'us', 'much', 'and/or',"wa","ha",",","n't" ,"”" ,"“" ,"’"
                           ]

In [ ]:



df_true= pd.read_csv("sample_data/true_news.csv")
df_fake=pd.read_csv("sample_data/fake_n.csv")
df_pants=pd.read_csv("sample_data/pants_on_fire.csv")

# New Section

In [ ]:

def make_dic():
  for stop in closed_class_stop_words:
    stop_word_dic[stop]=stop
            
make_dic()  

def clean_me(string):
  return remove_stop_words(word_tokenize(string))

def remove_stop_words(text):
  result=[]
  for word in text:
    word=word.lower()
    word=lemmatizer.lemmatize(word)
    if word in stop_word_dic:
      continue
    else:
      result.append(word)
  return result

In [ ]:
def clean_data_frameTF_IDF(df,AP,col="Statement"):
  text=df[col]
  clean_text=[]
  id=[]
  Link=[]
  df_clean=pd.DataFrame()
  for index, row in df.iterrows():
    text=row[col]
    res=" ".join(clean_me(text))
    clean_text.append(res)
    if col =="Statement":
      id.append(row['Date'])
      Link.append(row['Link'])
  if(AP):
      for index, row in df_pants.iterrows():
        text=row[col]
        res=" ".join(clean_me(text))
        clean_text.append(res)
        if col =="Statement":
          id.append(row['Date'])
          Link.append(row['Link'])

  if col =="Statement":
    #, "link":Link
    return pd.DataFrame({'Date': id, 'Clean_text': clean_text}),clean_text
  else:
    return pd.DataFrame({'Clean_tweet': clean_text})

df_clean_true=clean_data_frameTF_IDF(df_true,False)
df_clean_false=clean_data_frameTF_IDF(df_fake,True)


In [ ]:
def clean_data_frame(df,AP,col="Statement"):
  text=df[col]
  clean_text=[]
  id=[]
  Link=[]
  df_clean=pd.DataFrame()
  for index, row in df.iterrows():
    text=row[col]
    res=clean_me(text)
    clean_text.append(res)
    if col =="Statement":
      id.append(row['Date'])
      Link.append(row['Link'])
  
  if(AP):
    for index, row in df_pants.iterrows():
      text=row[col]
      res=clean_me(text)
      clean_text.append(res)
      if col =="Statement":
        id.append(row['Date'])
        Link.append(row['Link'])

  if col =="Statement":
    return pd.DataFrame({'Date': id, 'Clean_text': clean_text, "link":Link})
  else:
    return pd.DataFrame({'Clean_tweet': clean_text})

df_clean_true_list=clean_data_frame(df_true,False)
df_clean_false_list=clean_data_frame(df_fake,True)

In [ ]:
corpus=df_clean_false[1]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
np.random.seed(0)
X = csr_matrix(X)
svd = TruncatedSVD(n_components=1000, n_iter=10, random_state=42)
y=svd.fit_transform(X)

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.5, min_samples=5, metric='cosine').fit(y)

In [ ]:
# You can change the param to target different categories dates and use dates to identify topics that are fake 
#
#
#
from numpy.lib.function_base import disp
df=df_clean_false[0]
df=pd.concat([df,df["Date"].str.split(expand=True)], axis=1).rename(columns={0:"month",1:"day",2:'year'})
unique, counts = np.unique(clustering.labels_, return_counts=True)
counts_cluster=(dict(zip(unique, counts)))
sorted_dict = {}
sorted_keys = sorted(counts_cluster, key=counts_cluster.get, reverse=True) 
for w in sorted_keys:
    sorted_dict[w] = counts_cluster[w]
pd.options.display.max_colwidth = 200
for k,v in sorted_dict.items():
  if(v>=20):
    print(k," : ",v)
breaker=0
str_result=[]
d=df
dic_date={}
date=0
p=False
date_count={}
for i,x in enumerate(clustering.labels_):
  if(x==x):
    d=df.iloc[[i]]
    str_date=d["Date"].tolist()[0]
    if(len(d.loc[d["year"].str.contains(":")])==1):
      continue
    date=int(d["year"])
    Sep = d.loc[(d['Clean_text'].str.contains('krai'))]
    if(date in dic_date):
      if((len(Sep["Clean_text"])!=0) or p):
        dic_date[date].append(df_clean_false_list.iloc[i]['Clean_text'])
        if(date==2022):
          if(str_date in date_count):
            date_count[str_date]+=1
          else:
            date_count[str_date]=1   
    else:
      if(len(Sep["Clean_text"])!=0 or p):
        dic_date[date]=[(df_clean_false_list.iloc[i]['Clean_text'])]
        if(date==2022):
           if(str_date in date_count):
            date_count[str_date]+=1
           else:
            date_count[str_date]=1
    if(len(Sep["Clean_text"])!=0):
      pass
      #display(Sep)
    breaker+=1
    if(breaker>20000):
      break

print("printing")
for k,v in date_count.items():
  print(k,v)
  if (k[0:3]=="Mar"):
    print(k,v)

-1  :  4498
1  :  2498
4  :  59
12  :  41
5  :  30
30  :  23
42  :  23
24  :  20
printing
March 28, 2022 2
March 28, 2022 2
March 19, 2022 1
March 19, 2022 1
March 5, 2022 2
March 5, 2022 2
March 16, 2022 2
March 16, 2022 2
March 21, 2022 1
March 21, 2022 1
March 20, 2022 1
March 20, 2022 1
March 2, 2022 4
March 2, 2022 4
March 10, 2022 2
March 10, 2022 2
March 14, 2022 1
March 14, 2022 1
March 15, 2022 1
March 15, 2022 1
March 13, 2022 2
March 13, 2022 2
March 12, 2022 1
March 12, 2022 1
March 7, 2022 2
March 7, 2022 2
March 4, 2022 1
March 4, 2022 1
March 9, 2022 2
March 9, 2022 2
March 6, 2022 1
March 6, 2022 1
March 1, 2022 3
March 1, 2022 3
February 27, 2022 3
February 28, 2022 3
February 26, 2022 2
February 24, 2022 5
March 25, 2022 1
March 25, 2022 1
February 25, 2022 5
February 23, 2022 1
February 19, 2022 1
January 27, 2022 1
March 23, 2022 1
March 23, 2022 1
March 11, 2022 1
March 11, 2022 1


In [ ]:
for k,v in dic_date.items():
    print (k, len(list(filter(None, v))))

2022 53
2021 3
2020 5
2019 15
2016 1
2015 1
2014 3


In [ ]:
from collections import Counter
bag={}
all_p=True
res=" "
for k,v in dic_date.items():
  if(all_p):
    res=" "
  if(len(list(filter(None, v)))<30):
    continue
  for index in dic_date[k]:
    for el in index:
      if(el in bag):
        bag[el]+=1
      else:
        bag[el]=1
      res=res+" "+str(el)
  if(not all_p):
    print(res)
  words=res.split(' ')
  bigrams = zip(words, words[1:])
  bg=[]
  for w1 in bigrams:
    bg.append(w1[0]+"_"+w1[1])
  counts = Counter(bg)
  resy={}
  sorted_keys2 = sorted(bag, key=bag.get, reverse=True) 
  for w in sorted_keys2:
      resy[w] = bag[w]
  sorted_dict1={}
  sorted_keys1 = sorted(counts, key=counts.get, reverse=True) 
  for w in sorted_keys1:
      sorted_dict1[w] = counts[w]
  if(all_p):
    print(k," : ",resy)
    print(sorted_dict1)
if(not all_p):
  print(k," : ",resy)
  print(sorted_dict1)


2022  :  {'ukraine': 33, 'ukrainian': 20, 'russian': 15, 'video': 12, 'president': 7, 'photo': 7, 'russia': 6, 'soldier': 6, '2022': 6, 'volodymyr': 4, 'u.s.': 4, 'footage': 4, 'war': 4, 'kyiv': 4, 'troop': 4, 'staged': 3, 'invasion': 3, 'zelensky': 3, 'putin': 3, 'child': 3, 'tank': 3, 'fighter': 3, 'cnn': 2, 'shooting': 2, 'farmer': 2, 'army': 2, 'hillary': 2, 'combat': 2, 'wounded': 2, 'pilot': 2, 'operation': 2, 'biological': 2, 'weapon': 2, 'lab': 2, 'image': 2, 'country': 2, 'actor': 2, 'young': 2, 'captured': 2, 'face': 2, 'smith': 1, 'slapped': 1, 'chris': 1, 'rock': 1, 'attention': 1, 'edmonton': 1, 'canada': 1, 'claimed': 1, 'filmed': 1, 'zelenskyy': 1, 'singing': 1, 'imagine': 1, 'john': 1, 'lennon': 1, 'simpson': 1, 'predicted': 1, 'wore': 1, 'nazi': 1, 'cross': 1, 'symbol': 1, 'tweeted': 1, 'picture': 1, 'serbian': 1, 'resort': 1, 'resident': 1, 'donetsk': 1, 'lugansk': 1, 'wanted': 1, 'speak': 1, 'slowly': 1, 'starting': 1, 'build': 1, 'vow': 1, '‘': 1, 'crush': 1, 'traff